In [1]:
from parcels.numba.grid.field import NumbaField
from parcels.numba.grid.grid import Grid
from parcels.numba.grid.vector_field_2d import NumbaVectorField2D
from parcels.numba.grid.fieldset import NumbaFieldSet
from parcels.numba.grid.pset import create_record_pset
from parcels.application_kernels import AdvectionRK4
from numba import njit
import numpy as np

In [2]:
def create_grid(n_lon=20, n_lat=40, n_depth=5, n_time=10, field_3d=False):
    lon = np.linspace(1, 2, n_lon, dtype=np.float32)
    lat = np.linspace(0, 1, n_lat, dtype=np.float32)
    depth = np.linspace(2, 3, n_depth, dtype=np.float32)
    time = np.linspace(4, 5, n_time, dtype=np.float64)
    time_origin = 4
    if field_3d:
        data = np.random.randn(n_time, n_depth, n_lon, n_lat).astype(np.float32)
    else:
        data = np.random.randn(n_time, 1, n_lon, n_lat).astype(np.float32)
    return Grid.create_grid(lon, lat, depth, time, time_origin, mesh="linear"), data


In [3]:
pset = create_record_pset(100)

In [4]:
U = NumbaField(*create_grid())
V = NumbaField(*create_grid())

In [5]:
vec_field = NumbaVectorField2D("hi", U, V)
fieldset = NumbaFieldSet(U, V)

In [6]:
vec_field.eval(4.01, 2.5, 0.5, 1.5)

(-1.3146174120903051, -0.9744870647788072)

In [7]:
fieldset.UV[4.06, 2.5, 0.5, 1.5]

(-0.2968878555297889, -0.29795824587345365)

In [8]:
kernel = njit(AdvectionRK4)

In [9]:
p = pset[0]
p.depth = 2.5
p.lat = 0.5
p.lon = 1.5

In [10]:
kernel(p, fieldset, 4.01)

In [11]:
p

(0.01, 0.49093166, 1.4878715, 2.5, 0, 0)